<a href="https://colab.research.google.com/github/pshenoWasTaken/MIREA/blob/main/BD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%reload_ext sql
%sql sqlite:///store.db

## Задание 1
Вывести 5 самых больших по размеру (в байтах) треков жанра Blues и формата MPEG. Только название и размер.


In [23]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [25]:
%%sql
SELECT t.name, t.bytes
FROM tracks t
INNER JOIN genres g ON t.GenreId = g.GenreId
INNER JOIN media_types m ON t.MediaTypeId = m.MediaTypeId
WHERE g.name = 'Blues'
  AND m.name = 'MPEG audio file'
ORDER BY t.bytes DESC
LIMIT 5;



 * sqlite:///store.db
Done.


Name,Bytes
Old Love,15780747
Running On Faith,12536275
Lonely Stranger,10894406
Layla,9490542
Tears In Heaven,9032835


## Задание 2
Вывести названия всех треков и их исполнителей для всех треков жанра Rock, приобретенных сотрудниками Oracle.


In [28]:
%%sql
SELECT t.Name AS track_name, ar.Name AS artist_name
FROM tracks t
INNER JOIN genres g ON t.GenreId = g.GenreId
INNER JOIN invoice_items ii ON t.TrackId = ii.TrackId
INNER JOIN invoices i ON ii.InvoiceId = i.InvoiceId
INNER JOIN customers c ON i.CustomerId = c.CustomerId
INNER JOIN artists ar ON t.AlbumId = ar.ArtistId
WHERE g.Name = 'Rock'
  AND c.Company = 'Oracle';

 * sqlite:///store.db
Done.


track_name,artist_name
Sunday Bloody Sunday,"Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett"
New Year's Day,"Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett"
That's The Way,Stone Temple Pilots
Ten Years Gone,System Of A Down
Achilles Last Stand,"Terry Bozzio, Tony Levin & Steve Stevens"
Tea For One,"Terry Bozzio, Tony Levin & Steve Stevens"
No Quarter,The Clash
Bad Moon Rising,Green Day
Down On The Corner,Green Day
Up Around The Bend,Green Day


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций (если 0, можно не выводить).


In [30]:
%%sql
SELECT c.Company, COUNT(t.TrackId)
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
JOIN tracks t ON ii.TrackId = t.TrackId
JOIN genres g ON t.GenreId = g.GenreId
WHERE g.Name = 'Jazz'
  AND c.Company IS NOT NULL
GROUP BY c.Company
HAVING COUNT(t.TrackId) > 0;


 * sqlite:///store.db
Done.


Company,COUNT(t.TrackId)
Apple Inc.,2
BMW,1
Google Inc.,12
JetBrains s.r.o.,3
Microsoft Corporation,15
Oracle,3
SAP,1
Telus,2


## Задание 4
Вывести топ 3 плейлиста по продолжительности (продолжительность плейлиста = сумма длин треков в нём). Только название и продолжительность (в часах).


In [31]:
%%sql
SELECT pl.Name AS playlist_name, SUM(t.Milliseconds) / (1000.0 * 60 * 60) AS duration_hours
FROM playlists pl
JOIN playlist_track pt ON pl.PlaylistId = pt.PlaylistId
JOIN tracks t ON pt.TrackId = t.TrackId
GROUP BY pl.Name
ORDER BY duration_hours DESC
LIMIT 3;


 * sqlite:///store.db
Done.


playlist_name,duration_hours
From That TV Show,138.57329333333334
Hidden Gems,138.53488833333333
90’s Music,10.550303333333334


## Задание 5
Для каждого набора (жанр, тип медиа) вывести количество треков в них, причем вывести только те наборы, для которых **все** треки стоят больше 1$ и для которых в наборе есть хотя бы один трек.

In [32]:
%%sql
SELECT g.Name AS genre_name, m.Name AS media_type_name, COUNT(t.TrackId) AS track_count
FROM tracks t
JOIN genres g ON t.GenreId = g.GenreId
JOIN media_types m ON t.MediaTypeId = m.MediaTypeId
GROUP BY g.Name, m.Name
HAVING MIN(t.UnitPrice) > 1 AND COUNT(t.TrackId) > 0;



 * sqlite:///store.db
Done.


genre_name,media_type_name,track_count
Blues,AAC audio file,9
Comedy,MPEG audio file,17
Sci Fi & Fantasy,MPEG audio file,26
Science Fiction,MPEG audio file,13
TV Shows,MPEG audio file,93
